In [1]:
from elasticsearch import Elasticsearch
import urllib3
import warnings
from flatten_json import flatten
from helpers import LinkedList
import os


ELASTIC_PASS = os.environ["ES_PASS"]
NODES = [
    "https://esh1p:9200",
    "https://esh2p:9200",
    
]
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)


**Connect to Elasticsearch**

In [2]:
es = Elasticsearch(NODES,
    basic_auth=("elastic", ELASTIC_PASS),
    verify_certs=False                       
)


/home/filipzag/.local/lib/python3.9/site-packages/elasticsearch/_sync/client/__init__.py:395: SecurityWarning: Connecting to 'https://esh1p:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(
/home/filipzag/.local/lib/python3.9/site-packages/elasticsearch/_sync/client/__init__.py:395: SecurityWarning: Connecting to 'https://esh2p:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(


**Get data**

In [3]:
process_id = "8980"
index_pattern = "windows"
process_eid = "{f97bd2d6-b164-6393-040c-000000006c00}"
host_name = "WKSIBUDIMLIC.corp.diverto.hr"

q_str = 'event.code: 1 AND host.name: "{}" AND proces.entity_id: "{}"'.format(host_name, process_eid)


## Linked list of linked processes
linked_processes = LinkedList()


def  get_parent_process(process_eid):
    q_str = 'event.code: 1 AND host.name: "{}" AND process.entity_id: "{}"'.format(host_name, process_eid)
    
    response = es.search(index=index_pattern,
          query={"query_string": {
                   "query": q_str }},source = [ "process.name", "process.entity_id", "process.parent.entity_id"] 
          )
    
    if len(response['hits']['hits']) > 0:
        
        linked_processes.insert(response['hits']['hits'][0]['_source']['process']['name'])
        
        
        if response['hits']['hits'][0]['_source']['process']['parent']:
            
            get_parent_process(response['hits']['hits'][0]['_source']['process']['parent']['entity_id'])
        
        
        
get_parent_process(process_eid)

**Print process ancestry**

In [4]:
node = linked_processes.get_head()

while node != None:
    if node.get_next() == None:
        print(node.get_data(),end = "")
    else:
    
        print( node.get_data(),end = " => ")
    node = node.get_next()
    


smss.exe => smss.exe => wininit.exe => services.exe => svchost.exe => raserver.exe